#### Loading required libraries

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

#### Read the data

In [2]:
# Read in the data
clients = pd.read_csv('clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('payments.csv', parse_dates = ['payment_date'])


In [3]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [4]:
loans.sample(10)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
158,35089,credit,5732,0,11492,2006-03-04,2007-08-01,1.75
47,41480,credit,8624,1,11277,2011-11-09,2013-12-05,1.18
440,26945,other,9329,0,10154,2001-12-17,2004-07-22,5.65
314,49068,cash,11871,1,11154,2011-10-15,2013-11-07,4.08
52,41480,home,9766,0,10567,2008-06-03,2010-01-08,9.08
279,44601,home,11175,1,10721,2010-07-14,2012-11-06,7.14
119,39505,credit,7162,1,11052,2004-07-29,2006-06-24,8.60
430,26945,cash,9249,1,11482,2013-12-24,2016-05-11,2.86
19,46109,credit,559,1,10599,2008-02-15,2009-11-25,4.15
192,48177,credit,10830,0,11177,2009-06-07,2011-10-17,4.98


In [5]:
payments.sample(10)

,loan_id,payment_amount,payment_date,missed
1973,10988,1258,2011-10-03,1
1585,10624,1470,2001-06-24,0
3077,11855,128,2014-06-09,1
283,10302,522,2006-03-16,1
3432,10154,1647,2002-02-15,0
3333,11580,1228,2001-06-02,0
2150,11320,1217,2009-08-24,0
826,11975,692,2009-12-15,1
257,10425,1577,2002-01-31,0
1590,11882,445,2002-02-23,1


In [6]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


In [7]:
# Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [8]:
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.600000,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.300000,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.850000,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.850000,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.950000,13913,1212
5,39505,2011-10-14,153873,610,10,11.943883,7424.050000,14575,904
6,32726,2006-05-01,235705,730,5,12.370336,6633.263158,14802,851
7,35089,2010-03-01,131176,771,3,11.784295,6939.200000,13194,773
8,35214,2003-08-08,95849,696,8,11.470529,7173.555556,14767,667
9,48177,2008-06-09,190632,769,6,12.158100,7424.368421,14740,659


#### Entityset

In [9]:
es = ft.EntitySet(id = 'clients')

#### Entities

In [10]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [11]:
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [12]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [13]:
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

In [14]:
type(es)

featuretools.entityset.entityset.EntitySet

In [15]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [16]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

#### Relationships

In [17]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

In [18]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

#### Feature primitives

In [19]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,skew,aggregation,Computes the skewness of a data set.
1,last,aggregation,Returns the last value.
2,percent_true,aggregation,Finds the percent of 'True' values in a boolean feature.
3,time_since_last,aggregation,Time since last related instance.
4,median,aggregation,Finds the median value of any feature with well-ordered values.
5,any,aggregation,Test if any value is 'True'.
6,num_true,aggregation,Finds the number of 'True' values in a boolean.
7,std,aggregation,Finds the standard deviation of a numeric feature ignoring null values.
8,n_most_common,aggregation,Finds the N most common elements in a categorical feature.
9,trend,aggregation,Calculates the slope of the linear trend of variable overtime.


In [20]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
19,year,transform,Transform a Datetime feature into the year.
20,or,transform,"For two boolean values, determine if one value is 'True'."
21,months,transform,Transform a Timedelta feature into the number of months.
22,time_since_previous,transform,Compute the time since the previous instance.
23,days_since,transform,"For each value of the base feature, compute the number of days between it"
24,weeks,transform,Transform a Timedelta feature into the number of weeks.
25,cum_mean,transform,Calculates the mean of previous values of an instance for each value in a time-dependent entity.
26,is_null,transform,"For each value of base feature, return 'True' if value is null."
27,absolute,transform,Absolute value of base feature.
28,numwords,transform,Returns the words in a given string by counting the spaces.


In [21]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['years', 'month', 'subtract', 'divide'])

In [22]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
25707,10
26326,5
26695,8
26945,11
29841,8


In [23]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

,MEAN(payments.payment_amount)
client_id,
25707,1178.552795
26326,1166.736842
26695,1207.433824
26945,1109.473214
29841,1439.433333


In [24]:
features.head()

,income,credit_score,join_month,log_income,MEAN(loans.loan_amount),MEAN(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),LAST(loans.loan_type),LAST(loans.loan_amount),...,log_income / MEAN(loans.loan_amount),credit_score - join_month / income,income - log_income / credit_score,credit_score - log_income / LAST(payments.payment_amount),MEAN(loans.rate) / log_income,join_month / credit_score - join_month,income - join_month / income - log_income,join_month - credit_score / credit_score,log_income / LAST(payments.payment_amount),credit_score - log_income / MEAN(loans.loan_amount)
client_id,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,10,12.261611,7963.950000,3.477000,13913,9.44,home,2203,...,0.001540,0.002890,340.434361,2.525885,0.283568,0.016367,1.000011,-0.983897,0.050878,0.076437
26326,227920,633,5,12.336750,7270.062500,2.517500,13464,6.73,credit,5275,...,0.001697,0.002755,360.043702,0.665948,0.204065,0.007962,1.000032,-0.992101,0.013237,0.085372
26695,174532,680,8,12.069863,7824.722222,2.466111,14865,6.51,other,13918,...,0.001543,0.003850,256.646956,0.320504,0.204320,0.011905,1.000023,-0.988235,0.005792,0.085362
26945,214516,806,11,12.276140,7125.933333,2.855333,14593,5.65,cash,9249,...,0.001723,0.003706,266.133652,0.497009,0.232592,0.013836,1.000006,-0.986352,0.007687,0.111385
29841,38354,523,8,10.554614,9813.000000,3.445000,14837,6.76,home,7223,...,0.001076,0.013428,73.314427,0.640557,0.326398,0.015534,1.000067,-0.984704,0.013193,0.052221


In [25]:
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
25707,293.500000
26326,977.375000
26695,1769.166667
26945,1598.666667
29841,1125.500000
32726,799.500000
32885,1729.000000
32961,282.600000
35089,110.400000


#### Deep Feature Synthesis

In [26]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [27]:
features.iloc[:, 4:].head()

,SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),SKEW(loans.loan_amount),SKEW(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),...,NUM_UNIQUE(loans.WEEKDAY(loan_end)),MODE(loans.MODE(payments.missed)),MODE(loans.DAY(loan_start)),MODE(loans.DAY(loan_end)),MODE(loans.YEAR(loan_start)),MODE(loans.YEAR(loan_end)),MODE(loans.MONTH(loan_start)),MODE(loans.MONTH(loan_end)),MODE(loans.WEEKDAY(loan_start)),MODE(loans.WEEKDAY(loan_end))
client_id,,,,,,,,,,,,,,,,,,,,,
25707,159279,69.54,4149.486062,2.484186,13913,9.44,-0.186352,0.735470,1212,0.33,...,6,0,27,1,2010,2007,1,8,3,0
26326,116321,40.28,4393.666631,2.057142,13464,6.73,0.149658,1.181651,1164,0.50,...,5,0,6,6,2003,2005,4,7,5,2
26695,140845,44.39,4196.462499,1.561659,14865,6.51,0.168879,0.896574,2389,0.22,...,6,0,3,14,2003,2005,9,4,1,1
26945,106889,42.83,4543.621769,1.619717,14593,5.65,0.174492,-0.002227,653,0.13,...,6,0,16,1,2002,2004,12,5,0,1
29841,176634,62.01,4209.224171,2.122904,14837,6.76,-0.232215,0.055321,2778,0.26,...,7,1,1,15,2005,2007,3,2,5,1
